# Prepare data

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
url = 'https://shopee.co.th/api/v4/pages/get_category_tree'
category_list = requests.get(url).json()
product_class = pd.DataFrame.from_dict(category_list['data']['category_list'])[['catid', 'name']]
product_class = product_class.rename(columns={'catid':'product_class_id', 'name':'product_class_name'})
print("Total product_class :", product_class.shape[0])

product = pd.DataFrame(columns=['product_id', 'product_name', 'retail_price', 'product_class_id'])
for i in product_class['product_class_id'] :
    url = f"https://shopee.co.th/api/v4/recommend/recommend?bundle=category_landing_page&cat_level=1&catid={i}&limit=60&offset=0"
    data = pd.DataFrame.from_dict(requests.get(url).json()['data']['sections'][0]['data']['item'])[['name', 'price']]
    data['price'] = data['price'].astype('str').str.replace("00000", "").astype('int')
    data['product_class_id'] = i
    data = data.reset_index()
    data = data.rename(columns={'index':'product_id', 'name':'product_name' ,'price':'retail_price'})
    product = pd.concat([product, data], ignore_index=True)
print("Total product :", product.shape[0])

sales_transaction = pd.DataFrame(columns=['transaction_id', 'product_id', 'quantity'])
for i in range(1, 100) :
    no_item_bucket = np.random.randint(1,7)
    product_id = np.random.choice(product.shape[0], no_item_bucket)
    quantity = np.random.randint(1,10, no_item_bucket)
    transaction = pd.DataFrame(zip(product_id, quantity), columns=['product_id', 'quantity'])
    transaction['transaction_id'] = i
    sales_transaction = pd.concat([sales_transaction, transaction], ignore_index=True)
print("Total sales_transaction :", sales_transaction.shape[0])


Total product_class : 24
Total product : 1413
Total sales_transaction : 361


In [3]:
sales_transaction.head(10)

,transaction_id,product_id,quantity
0,1,1002,4
1,1,205,8
2,1,722,5
3,2,683,5
4,2,1318,9
5,2,108,2
6,3,310,6
7,3,1069,6
8,4,1072,9
9,4,1003,2


In [4]:
product.head(10)

,product_id,product_name,retail_price,product_class_id
0,0,BK529-LONG เสื้อยอดฮิต แจ็คเก็ต เเสื้อกันหนา...,20,11044945
1,1,ใหม่ เสื้ออภิชาติฟาร์ม ผ้ามันแขนสั้น เสื้อยืดเ...,108,11044945
2,2,กางขากระบอกใหญ่ Lazy ขาเริ่มต้นที่9,1050,11044945
3,3,กางเกงขาสั้นชิโน่ ผ้า Cotton100% นิ่มใส่สบาย b...,279,11044945
4,4,BK515-LONG เสื้อยอดฮิต แจ็คเก็ต เเสื้อกันหนา...,20,11044945
5,5,(ลูกค้าใหม่ 1 บาท) เสื้อ+กางเกงขาสั้น ผ้าเบา ไ...,128,11044945
6,6,LOOKER - กางเกงChino ทรงกระบอกเล็ก,290,11044945
7,7,กางเกงขาสั้น กางเกงขาสั้นลำลอง แห้งเร็ว กางเกง...,60,11044945
8,8,กางเกงในชาย กางเกงในผู้ชาย แพ็ค10ตัว M L XL,153,11044945
9,9,S-23 เสื้อยืด คอกลมสีพื้นปักมี 15สี M-XL,68,11044945


In [5]:
product_class.head(10)

,product_class_id,product_class_name
0,11044945,Men Clothes
1,11044944,Women Clothes
2,11044951,Mobile & Gadgets
3,11044959,Beauty & Personal Care
4,11044942,Health & Wellness
5,11044960,Baby & Toys
6,11044952,Watches & Glasses
7,11044956,Home & Living
8,11044949,Men Shoes
9,11044964,Bags


In [6]:
import sqlite3
connect = sqlite3.connect('ecommerce.db')
sales_transaction.to_sql("sales_transaction", connect, if_exists='replace', index=False)
product.to_sql("product", connect, if_exists='replace', index=False)
product_class.to_sql("product_class", connect, if_exists='replace', index=False)

24

# SQL

In [7]:
%load_ext sql
%sql sqlite:///ecommerce.db

In [8]:
%%sql
SELECT * FROM sales_transaction LIMIT 10

 * sqlite:///ecommerce.db
Done.


transaction_id,product_id,quantity
1,1002,4
1,205,8
1,722,5
2,683,5
2,1318,9
2,108,2
3,310,6
3,1069,6
4,1072,9
4,1003,2


In [9]:
%%sql
SELECT * FROM product LIMIT 10

 * sqlite:///ecommerce.db
Done.


product_id,product_name,retail_price,product_class_id
0,BK529-LONG เสื้อยอดฮิต แจ็คเก็ต เเสื้อกันหนาว เสื้อฮู้ดแขนยาวเสื้อผ้าแฟชั่นนานาชาต***ปลีก-ส่ง***,20,11044945
1,ใหม่ เสื้ออภิชาติฟาร์ม ผ้ามันแขนสั้น เสื้อยืดเสื้อ อภิชาติฟาร์ม เเท้ เเขนสั้น สีดำ/ขาว 💢💢เสื้อจากงาน NGO มีของพร้อมส่ง,108,11044945
2,กางขากระบอกใหญ่ Lazy ขาเริ่มต้นที่9,1050,11044945
3,กางเกงขาสั้นชิโน่ ผ้า Cotton100% นิ่มใส่สบาย by Normal Boy,279,11044945
4,BK515-LONG เสื้อยอดฮิต แจ็คเก็ต เเสื้อกันหนาว เสื้อฮู้ดแขนยาวเสื้อผ้าแฟชั่นนานาชาต***ปลีก-ส่ง***,20,11044945
5,(ลูกค้าใหม่ 1 บาท) เสื้อ+กางเกงขาสั้น ผ้าเบา ไม่หด ไม่ย้วย เสื้อยืดคอกลมสีพื้น ชุดกีฬาผู้ชาย ชุด,128,11044945
6,LOOKER - กางเกงChino ทรงกระบอกเล็ก,290,11044945
7,กางเกงขาสั้น กางเกงขาสั้นลำลอง แห้งเร็ว กางเกงขาสั้น เอวยางยืดคุณภาพดีและราคาถูก ขายร้อนในญี่ปุ่น เกาหลี,60,11044945
8,กางเกงในชาย กางเกงในผู้ชาย แพ็ค10ตัว M L XL,153,11044945
9,S-23 เสื้อยืด คอกลมสีพื้นปักมี 15สี M-XL,68,11044945


In [10]:
%%sql
SELECT * FROM product_class LIMIT 10

 * sqlite:///ecommerce.db
Done.


product_class_id,product_class_name
11044945,Men Clothes
11044944,Women Clothes
11044951,Mobile & Gadgets
11044959,Beauty & Personal Care
11044942,Health & Wellness
11044960,Baby & Toys
11044952,Watches & Glasses
11044956,Home & Living
11044949,Men Shoes
11044964,Bags


In [11]:
%%sql
WITH sum_transaction AS (
SELECT product_class_id
, product_name
, sum(quantity * retail_price) as sales_value
, ROW_NUMBER() OVER(PARTITION BY product_class_id ORDER BY sum(quantity * retail_price) DESC, quantity DESC) as rank
FROM sales_transaction a
INNER JOIN product b
ON a.product_id = b.product_id
WHERE a.product_id IS NOT NULL
GROUP BY product_class_id
, product_name
)

SELECT a.product_class_id
, product_class_name
, rank
, product_name
, sales_value
FROM sum_transaction a
INNER JOIN product_class b
ON a.product_class_id = b.product_class_id
WHERE rank <= 2
ORDER BY a.product_class_id, sales_value DESC

 * sqlite:///ecommerce.db
Done.


product_class_id,product_class_name,rank,product_name,sales_value
11044942,Health & Wellness,1,Sinocare เครื่องตรวจวัดระดับน้ำตาลในเลือด เครื่องตรวจน้ำตาล(เบาหวาน) รุ่น Safe Accu เครื่องตรวจ+เข็มเจาะ+แผ่น แม่นยำ100%,2241
11044942,Health & Wellness,2,【ซึมเร็วสุด!！】ไม่รู้สึก ผ้าอ้อมผู้ใหญ่ Andlove ใช้ได้ทั้งชายหญิง ปลี่ยนง่าย M/L/XL(10 Pcs/1 Pack) adult diapers pants,936
11044943,Food & Beverages,1,[15 Feb ลดเหลือ 9 .-] Wel-B Freeze-dried Strawberry 14g (สตรอเบอรี่กรอบ 14g. ตราเวลบี) 1 ซอง 33 บาท - ขนม,5994
11044943,Food & Beverages,2,ถั่วพิสตาชิโอ้เกรดเอ(U.S.A) 500กรัม,1107
11044944,Women Clothes,1,"🔥G50🔥เอว 25-39"" กางเกงเอวสูงไร้ขอบ ซิปข้าง ยาว39"" กางเกงผ้าโรเชฟแท้ เนื้อดี ขาเดฟ9ส่วน ใส่ทำงาน เหมาะสำหรับสาวไซส์ S-7XL",1773
11044944,Women Clothes,2,กางเกงทรงบอย เอวสูง ผ้าเปเป้ 👗งานป้าย Rainbow Shop💕 กางเกงขากระบอกเล็กทรงบอย 9 ส่วน(BB),1701
11044945,Men Clothes,1,กางเกงขายาวชิโน่ ผ้ายืด ทรงกระบอกเล็ก (Slim-fit) by Normal Boy,3510
11044945,Men Clothes,2,กางขากระบอกใหญ่ Lazy ขาเริ่มต้นที่9,3150
11044946,Home Entertainment,1,[ผ่อน 0%] AIWA Retro Bluetooth Speaker ลำโพงบลูทูธพกพา BASS++,5780
11044946,Home Entertainment,2,SHURE UR01/UR02 ไมโครโฟนไร้สาย 2 ไมโครโฟนแบบใช้มือถือ 50M ระยะทางรับ UHF FM Cyclic ชาร์จไม่มีการรบกวน KTV เวทีวงดนตรีป,1794
